# INITIALIZATION

This cell does the following things:
 - initializes engine and modules
 - prompts player to select their color, and make first move if white
 - engine makes its first move as well, first if white or after player if black
 - displays board states and engine's preferences

In [ ]:
import SmoothBrainChess
import chess, chess.svg
from IPython.display import HTML, display

def game_is_over( current_board ):
    if current_board.is_checkmate():
        print('CHECKMATE, GAME OVER!')
        return True
    if current_board.is_stalemate():
        print('STALEMATE!')
        return True
    if current_board.has_insufficient_material(chess.WHITE) and current_board.has_insufficient_material(chess.BLACK):
        print('TIE BY INSUFFICIENT MATERIAL!')
        return True
    return False


board = chess.Board()
print('STARTING BOARD: ')
display(chess.svg.board(board))
user_color = input('Select Player Color: ("W" or "B")')

# SELECT COLOR AND MAKE FIRST MOVE IF WHITE
if user_color.upper() == 'W':
    engine_color = 'BLACK'
    print('User has selected WHITE')
    
    legal_move = False
    while not legal_move:
        player_move = input('Enter player move in UCI format (eg "e2e4"):')
        if player_move.upper() == 'QUIT':
            break
        elif player_move in [mv.uci() for mv in board.legal_moves]:
            board.push( chess.Move.from_uci(player_move) )
            print('PLAYER FIRST MOVE:')
            display(chess.svg.board(board))
            legal_move = True
        
        
        else:
            print('Illegal move!')
    
elif user_color.upper() == 'B':
    engine_color = 'WHITE'
    print('User has selected BLACK')
else:
    print('User color not recognized!')
    quit()

# INITIALIZE ENGINE
sbcEngine = SmoothBrainChess.SBCEngine(2000,'model_weights_2k_3lin_do.pth','Output_Node_Key.csv',engine_color)

# ENGINE'S TURN
moves_legal,values_legal,model_out = sbcEngine.topk( board, 5 , True)

# SHOW THE ENGINE THINKING!
base_color = '#00F0FF'
arrows = []
for ind, mv in enumerate(moves_legal):
    col = base_color
    col = col[0:3]+chr(ord(col[3])-(ind+1))+col[4:]

    if mv == 'O-O':
        if engine_color == 'WHITE':
            mv = 'e1g1'
            moves_legal[ind] = mv
        if engine_color == 'BLACK':
            mv = 'e8g8'
            moves_legal[ind] = mv
    elif mv == 'O-O-O':
        if engine_color == 'WHITE':
            mv = 'e1c1'
            moves_legal[ind] = mv
        if engine_color == 'BLACK':
            mv = 'e8c8'
            moves_legal[ind] = mv

    arrows.append(chess.svg.Arrow( getattr(chess,mv[0:2].upper()) , getattr(chess,mv[2:].upper()),color=col))
print('ENGINE THINKING:')
display(chess.svg.board(board,arrows=arrows))

# MAKE THE ENGINE'S MOVE
print('ENGINE FIRST MOVE: '+moves_legal[0])
board.push( chess.Move.from_uci( moves_legal[0]))
display(chess.svg.board(board))


# PLAY CHESS!

This cell works as follows:
 - At end of previous cell, it was the engine's turn
 - each run of this cell is a turn sequence
 - player is to specify their desired move at the top of this cell in UCI notation
 - player move is made, board is printed
 - engine is run, thought process is displayed, and board is printed (three board prints per run of cell)
 - then, to make next move, player changes their move at top of cell, and re-runs cell

Done this way because board graphics updates inside loop don't allow for resizing of board and it's hard to see what's going on


In [ ]:
player_move = 'a2a3' # PLAYER MOVE GOES HERE IN UCI NOTATION!

# CHECK LEGALITY OF PLAYER MOVE
if player_move not in [mv.uci() for mv in board.legal_moves]:
    print('Illegal move!')
    quit()

# MAKE THE PLAYER'S MOVE
print('PLAYER MOVE: '+player_move)
board.push( chess.Move.from_uci( player_move ))
display(chess.svg.board(board))

# CHECK FOR END CONDITIONS
if game_is_over( board ):
    quit()

# ENGINE'S TURN
moves_legal,values_legal,model_out = sbcEngine.topk( board, 5 , True)

# SHOW THE ENGINE THINKING!
base_color = '#00F0FF'
arrows = []
for ind, mv in enumerate(moves_legal):
    col = base_color
    col = col[0:3]+chr(ord(col[3])-(ind+1))+col[4:]

    # CONVERT CASTLING NODES TO UCI BASED ON ENGINE COLOR
    if mv == 'O-O':
        if engine_color == 'WHITE':
            moves_legal[ind] = 'e1g1'
        if engine_color == 'BLACK':
            moves_legal[ind] = 'e8g8'
    elif mv == 'O-O-O':
        if engine_color == 'WHITE':
            moves_legal[ind] = 'e1c1'
        if engine_color == 'BLACK':
            moves_legal[ind] = 'e8c8'
    
    # CREATE ARROW OBJECTS BASED ON ENGINE'S FAVORITE MOVES
    arrows.append(chess.svg.Arrow( getattr(chess,moves_legal[ind][0:2].upper()) , getattr(chess,moves_legal[ind][2:].upper()),color=col))
    
print('ENGINE THINKING:')
display(chess.svg.board(board,arrows=arrows))

# MAKE THE ENGINE'S MOVE
print('ENGINE MOVE: '+moves_legal[0])
board.push( chess.Move.from_uci( moves_legal[0]))
display(chess.svg.board(board))

# CHECK FOR END CONDITIONS
if game_is_over( board ):
    quit()
else:
    print('PLAYER TURN!')